# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique:
* Model:
* Evaluation approach:
* Fine-tuning dataset:

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
!pip install pip install datasets scikit-learn peft

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from datasets import load_dataset
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

In [3]:
# I chose spam classifcation -> 2 labels + dicts (for readability as it was in the exmaples)
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0:'NOT SPAM', 1:'SPAM'},
    label2id = {"NOT SPAM": 0, 'SPAM':1})

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
data_set = load_dataset("sms_spam", split='train').train_test_split(
shuffle=True, seed=123, test_size=0.2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for sms_spam contains custom code w

In [4]:
model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [5]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [6]:
data_set['test']

Dataset({
    features: ['sms', 'label'],
    num_rows: 1115
})

In [7]:
splits=['train','test']

In [8]:
## Firstly tokenizing
data_tokenized = {}
for split in splits:
    data_tokenized[split] = data_set[split].map(lambda x: tokenizer( x['sms'],truncation=True,
                                      padding='max_length',  return_tensors="pt"), batched=True)
    data_tokenized[split].set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

In [9]:
batch_size = 2
train_loader = DataLoader(data_tokenized['train'], batch_size=batch_size)
test_loader = DataLoader(data_tokenized['test'], batch_size=batch_size)

In [17]:
def evaluate_results(model_, data_loader):
  predictions = []
  labels = []
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model_ = model_.to(device)
  with torch.no_grad():
      for batch in tqdm(data_loader):
          input_ids = batch['input_ids']
          attention_mask = batch['attention_mask']
          input_ids =input_ids.to(device)
          attention_mask =attention_mask.to(device)
          outputs = model_(input_ids=input_ids, attention_mask=attention_mask)
          prediction = torch.argmax(outputs.logits, dim=-1)

          if device !='cpu':
              prediction =prediction.to('cpu')
          prediction = prediction.numpy()
          label = batch['label'].numpy()

          labels.extend(label)
          predictions.extend(prediction)
      return accuracy_score(labels,predictions)


In [18]:
evaluate_results(model,test_loader)

100%|██████████| 558/558 [00:18<00:00, 29.93it/s]


0.37488789237668163

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

Now I will try approach with Trainer


In [69]:
from peft import LoraConfig,TaskType
from peft import get_peft_model
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
import numpy as np

In [77]:

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0:'NOT SPAM', 1:'SPAM'},
    label2id = {"NOT SPAM": 0, 'SPAM':1})


config  = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=4,
    target_modules=['classifier']
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [78]:
lora_model = get_peft_model(model=model,peft_config=config)

In [79]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": accuracy_score(labels,predictions)}



In [81]:
trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./results",
        learning_rate=0.001,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        eval_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=1,
        weight_decay=0.01,
        load_best_model_at_end=True,
        label_names=["labels"]
    ),
    train_dataset=data_tokenized["train"],
    eval_dataset=data_tokenized["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics

)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.061000,0.030158,0.992825


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=1115, training_loss=0.07259597436194998, metrics={'train_runtime': 101.1364, 'train_samples_per_second': 44.089, 'train_steps_per_second': 11.025, 'total_flos': 598867535745024.0, 'train_loss': 0.07259597436194998, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

In [54]:
trainer.evaluate()['eval_accuracy']

0.9937219730941704

Check using the old function - for curiosity reason

In [47]:
evaluate_results(lora_model,test_loader)

100%|██████████| 558/558 [00:17<00:00, 31.01it/s]


0.9937219730941704

In [82]:
lora_model.save_pretrained("my_lora_test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [84]:
from peft import AutoPeftModelForSequenceClassification, PeftModel

base_model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0:'NOT SPAM', 1:'SPAM'},
    label2id = {"NOT SPAM": 0, 'SPAM':1})
lora_model1 = AutoModelForSequenceClassification.from_pretrained('my_lora_test')
# lora_model1= lora_model1.merge_and_unload()
# lora_model1 = AutoPeftModelForSequenceClassification.from_pretrained("my_lora")
# lora_model1.eval()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: Target modules {'classifier'} not found in the base model. Please check the target modules and try again.

In [ ]:
evaluate_results(lora_model1,test_loader)
